# Ensemble Learning and Random Forest

## Vote based Classifier

In [1]:
# Even each Classifier is weak learner, combining each classifier's result make strong learner

In [2]:
# Based on law of Large Number

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

x, y= make_moons(n_samples=100, noise=0.15, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.25)

In [4]:
len(y_train)

75

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

volt_clf = VotingClassifier(
        estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
        voting='hard'
)

volt_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [6]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.88
RandomForestClassifier 1.0
SVC 1.0


## Bagging(bootstrap aggregation) and pasting 

In [7]:
# Bagging -> allow overlap when trainset sampling
# pasting -> deny overlap when trainset sampling

### Using sckit learn for bagging and pasting

In [8]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators= 500,
    bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(x_train, y_train)
y_pred = bag_clf.predict(x_test)

In [9]:
accuracy_score(y_test, y_pred)

1.0

In [10]:
bag_clf.oob_score_

0.9333333333333333

In [11]:
bag_clf.oob_decision_function_

array([[0.25414365, 0.74585635],
       [0.06321839, 0.93678161],
       [1.        , 0.        ],
       [0.18716578, 0.81283422],
       [0.97752809, 0.02247191],
       [0.99481865, 0.00518135],
       [0.04040404, 0.95959596],
       [0.75789474, 0.24210526],
       [0.5047619 , 0.4952381 ],
       [0.        , 1.        ],
       [0.05555556, 0.94444444],
       [0.        , 1.        ],
       [0.9017341 , 0.0982659 ],
       [0.78609626, 0.21390374],
       [0.90291262, 0.09708738],
       [0.88826816, 0.11173184],
       [0.02054795, 0.97945205],
       [0.23756906, 0.76243094],
       [0.45698925, 0.54301075],
       [1.        , 0.        ],
       [0.01020408, 0.98979592],
       [0.        , 1.        ],
       [0.96315789, 0.03684211],
       [0.01041667, 0.98958333],
       [1.        , 0.        ],
       [0.99456522, 0.00543478],
       [0.01030928, 0.98969072],
       [1.        , 0.        ],
       [0.06470588, 0.93529412],
       [0.1741573 , 0.8258427 ],
       [0.

## Random Patch and Random subspace

In [12]:
# max_feature, bootstrap_feature -> feature sampling
# max_sample, bootstrap -> sample sampling
# feature sampling make various estimator and increase bias and decrease variance

## Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators= 500, max_leaf_nodes= 16, n_jobs=-1)
rnd_clf.fit(x_train, y_train)

y_pred_rf = rnd_clf.predict(x_test)

In [14]:
accuracy_score(y_test, y_pred_rf)

1.0

In [15]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
    n_estimators= 500, max_samples=1.0, bootstrap=True, n_jobs=-1)

### Extra Tree

In [16]:
# more random tree
# don't find optimized threshold
# faster than random forest

### Find feature importance

In [17]:
from sklearn.datasets import load_iris 

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators= 500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in  zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09446854862189182
sepal width (cm) 0.023526267179068888
petal length (cm) 0.4051396116804017
petal width (cm) 0.4768655725186373


## Boosting

### AdaBoost(Adaptive Boosting)

In [18]:
# Idea of Adaboost -> Underfitted Dataset's weight ↑
# -> It can be learning about difficult to train dataset 

In [19]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators= 200,
    algorithm="SAMME.R", learning_rate=0.5
)
ada_clf.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=None)

In [22]:
ada_clf_pred = ada_clf.predict(x_test)

In [24]:
accuracy_score(ada_clf_pred, y_test)

1.0

## Gradient Boosting

In [26]:
# instead modifing previous sample weight, train residual error 

In [28]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor()
tree_reg1.fit(x_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [30]:
y2_train = y_train - tree_reg1.predict(x_train)
tree_reg2 = DecisionTreeRegressor()
tree_reg2.fit(x_train, y2_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [31]:
y3_train = y2_train - tree_reg2.predict(x_train)
tree_reg3 = DecisionTreeRegressor()
tree_reg3.fit(x_train, y3_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [33]:
y_pred = sum(tree.predict(x_train) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [36]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=1.0, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=3, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [37]:
# if set learning rate low -> running  time ↑, predict error ↓

In [42]:
import numpy as np
from sklearn.metrics import mean_squared_error

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators= 120)
gbrt.fit(x_train, y_train)

errors =[
    mean_squared_error(y_test, y_pred)
    for y_pred in gbrt.staged_predict(x_test)
]

In [43]:
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=61, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [45]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(x_train, y_train)
    y_pred = gbrt.predict(x_test)
    val_error = mean_squared_error(y_test, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break

## Stacking

In [46]:
# It seems like NN